In [1]:
import pandas as pd

df = pd.read_json("../../datas/nuclei/detected.json", lines=True)

df.head(5)

,method,path,headers,uuid,peer,status,cookies,response_msg,timestamp,post_data
0,GET,/,"{'host': '127.0.0.1', 'user-agent': 'curl/7.81...",4d001351-3ab5-4ebe-b891-97c5ef3d1bac,"{'ip': '172.18.0.1', 'port': 47064}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-17 11:50:40.532834,NaN
1,HEAD,/,"{'host': '127.0.0.1', 'user-agent': 'Mozilla/5...",4d001351-3ab5-4ebe-b891-97c5ef3d1bac,"{'ip': '172.18.0.1', 'port': 58566}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-17 11:51:44.945677,NaN
2,GET,/servlet/codesettree?flag=c&status=1&codesetid...,"{'host': '127.0.0.1', 'user-agent': 'Mozilla/5...",4d001351-3ab5-4ebe-b891-97c5ef3d1bac,"{'ip': '172.18.0.1', 'port': 58590}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-17 11:51:45.022008,NaN
3,POST,/WEB_VMS/LEVEL15/,"{'host': '127.0.0.1', 'user-agent': 'Mozilla/5...",4d001351-3ab5-4ebe-b891-97c5ef3d1bac,"{'ip': '172.18.0.1', 'port': 58584}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-17 11:51:45.028476,{}
4,POST,/zentao/user-login.html,"{'host': '127.0.0.1', 'user-agent': 'Mozilla/5...",4d001351-3ab5-4ebe-b891-97c5ef3d1bac,"{'ip': '172.18.0.1', 'port': 58654}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-17 11:51:45.096028,"{'account': 'admin' and updatexml(1,concat(0x..."


In [2]:
methods_matrix = []
tmp = []
sess_uuids_methods = {
  # "{sess_uuid}": []
}
sess_uuids = []

for _, row in df.iterrows():
  sess_uuid = row.get("cookies", {}).get("sess_uuid") or row.get("response_msg", {}).get("response", {}).get("message", {}).get("sess_uuid")
  method = row["method"]

  if sess_uuid in sess_uuids:
    sess_uuids_methods[sess_uuid].append(method)
  else:
    sess_uuids_methods[sess_uuid] = [method]
    sess_uuids.append(sess_uuid)

methods_matrix = list(sess_uuids_methods.values())
# for paths in sess_uuids_methods.values():
#   methods_matrix.append(paths)

sess_uuids_methods
# methods_matrix
# print(len(methods_matrix), len(sess_uuids_methods.keys()))

{'5c03d747-efc4-417c-9ab2-87a504285fb5': ['GET'],
 'a7eb06e3-388a-436c-af5c-1778a2b34e3d': ['HEAD',
  'POST',
  'GET',
  'GET',
  'POST',
  'GET',
  'GET',
  'GET'],
 '481b180f-6499-4636-a881-d1b5948991e4': ['GET',
  'GET',
  'GET',
  'GET',
  'POST',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET'],
 'aa449b70-96de-4a22-8bbb-816a1491958e': ['POST',
  'GET',
  'POST',
  'GET',
  'GET',
  'GET',
  'GET',
  'POST'],
 'e9c12af5-5288-43d8-a728-60deda9557dd': ['POST',
  'POST',
  'POST',
  'GET',
  'GET',
  'GET',
  'GET',
  'POST',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET'],
 'eb6f4177-e8a3-4a3e-a797-9d3b250d9b01': ['POST', 'GET', 'GET'],
 'a7da2687-0745-4aa9-885d-d4d76766904d': ['POST',
  'POST',
  'POST',
  'POST',
  'GET',
  'GET'],
 'a969159d-11d9-4466-b89b-b3cc8da3df27': ['GET',
  'POST',
  'POST',
  'GET',
  'POST',
  'POST',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET'],
 '68dcbc12-f6da-4111-9d7f-7083a926724d': ['POST',
  'POST',
  'GET',
  'GET',
  

In [3]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth

te = TransactionEncoder()
te_ary = te.fit(methods_matrix).transform(methods_matrix)
df = pd.DataFrame(te_ary, columns=te.columns_)

df

,DELETE,GET,HEAD,OPTIONS,PATCH,POST,PUT,TRACE
0,False,True,False,False,False,False,False,False
1,False,True,True,False,False,True,False,False
2,False,True,False,False,False,True,False,False
3,False,True,False,False,False,True,False,False
4,False,True,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...
922,False,True,False,False,False,False,False,False
923,False,True,False,False,False,False,False,False
924,False,True,False,False,False,False,False,False
925,False,True,False,False,False,False,False,False


In [8]:
# frequent_itemsets = fpgrowth(df, min_support=0.3, use_colnames=True)
frequent_itemsets = fpgrowth(df, min_support=0.00001, use_colnames=True)
### alternatively:
#frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
#frequent_itemsets = fpmax(df, min_support=0.6, use_colnames=True)

frequent_itemsets.head(20)

,support,itemsets
0,0.981661,(GET)
1,0.760518,(POST)
2,0.002157,(HEAD)
3,0.029126,(PUT)
4,0.008630,(DELETE)
5,0.004315,(OPTIONS)
6,0.003236,(PATCH)
7,0.001079,(TRACE)
8,0.744337,"(GET, POST)"
9,0.002157,"(HEAD, POST)"


In [9]:
import psycopg2

conn = psycopg2.connect(database="web_honeypot_nuclei", user = "postgres", password = "admin", host = "127.0.0.1", port = "5432")

print("Opened database successfully")

Opened database successfully


In [10]:
# create table
cur = conn.cursor()
cur.execute('''CREATE TABLE assoc_rules_methods (
            ID INT PRIMARY KEY     NOT NULL,
            SUPPORT           REAL    NOT NULL,
            METHOD            VARCHAR(255)     NOT NULL);''')

print("Table created successfully")

conn.commit()

Table created successfully


In [11]:
# insert data

cur = conn.cursor()

insert_query = """
    INSERT INTO assoc_rules_methods (ID, SUPPORT, METHOD)
    VALUES (%s, %s, %s)
"""

for idx, row in frequent_itemsets.iterrows():
    cur.execute(
        insert_query,
        (int(idx), float(row["support"]), str(list(row["itemsets"])))
    )

conn.commit()
print("Records created successfully")
conn.close()

Records created successfully
